In [11]:
import pandas as pd
import ijson
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [5]:
time = []
text = []
city = []
location =[]
# with open(f’{tmp}_data.json’, ‘r’, encoding=’utf-8’) as f:
with open('data.json', 'r') as f:
    objects = ijson.items(f, 'results.item')
    for row in objects:
        if row['text'].startswith("RT") == False:
            city.append(row['place']['name'])
            location.append(row['coordinates'])
            time.append(row['created_at'])
            text.append(row['text'])

data = pd.DataFrame({"text":text,"time":time,"city":city,"location":location},columns=["text","time","city","location"])
data['time'] = pd.to_datetime(data['time']).dt.date

In [6]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

In [26]:
print(predictor.predict("sad!"))
print(predictor.predict("ok"))
print(predictor.predict("happy"))

[2021-04-27 17:28:01,816] [WARNING] [W108] The rule-based lemmatizer did not find POS annotation for the token 'sad'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[2021-04-27 17:28:01,817] [WARNING] [W108] The rule-based lemmatizer did not find POS annotation for the token '!'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[2021-04-27 17:28:01,832] [WARNING] [W108] The rule-based lemmatizer did not find POS annotation for the token 'ok'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[2021-04-27 17:28:01,859] [WARNING] [W108] The rule-based lemmatizer did not find POS annotation for the token 'happy'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.


{'logits': [-1.4538416862487793, 1.420343041419983], 'probs': [0.05344456434249878, 0.9465554356575012], 'token_ids': [419, 589], 'label': '0', 'tokens': ['sad', '!']}
{'logits': [3.149426221847534, -3.0729405879974365], 'probs': [0.9980193376541138, 0.0019806120544672012], 'token_ids': [1], 'label': '1', 'tokens': ['@@UNKNOWN@@']}
{'logits': [4.94352912902832, -4.841168403625488], 'probs': [0.999943733215332, 5.630350278806873e-05], 'token_ids': [1036], 'label': '1', 'tokens': ['happy']}


In [24]:
df = spark.createDataFrame(data)
# spacy.load('en_core_web_sm')
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")
def AllanNLP(text):
    return predictor.predict(text)['label']
sentiment_analysis_udf = udf(AllanNLP)
score = sentiment_analysis_udf( df['text'] )

df  = df.withColumn("sentiment_score", score)
df.show()

+--------------------+----------+---------+--------+---------------+
|                text|      time|     city|location|sentiment_score|
+--------------------+----------+---------+--------+---------------+
|Brilliant thread....|2020-08-31|Melbourne|    null|              1|
|@TimWilsonMP Have...|2020-08-31|Melbourne|    null|              1|
|@Rdene915 @Christ...|2020-08-31|Melbourne|    null|              1|
|A photo of you in...|2020-08-31|Melbourne|    null|              1|
|@IndyCat14 @JoyOf...|2020-08-31|Melbourne|    null|              1|
|@BuckleyIOP It re...|2020-08-31|Melbourne|    null|              1|
|@kruevans But you...|2020-08-31|Melbourne|    null|              1|
|@johniadarola I d...|2020-08-31|Melbourne|    null|              1|
|My sister turns 7...|2020-08-31|Melbourne|    null|              1|
|@kruevans It’s sh...|2020-08-31|Melbourne|    null|              1|
|@runwader Cannot ...|2020-08-31|Melbourne|    null|              1|
|@abcmelbourne mor...|2020-08-31|M

In [ ]:
def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())
df  = df.withColumn("sentiment", sentiment_udf( df['sentiment_score'] ))
SA_results = df.select('text','time','sentiment_score','sentiment')